In [2]:
def fill_template(template, pronoun, profession):
    template = template.replace('PROFESSION',profession)
    template = template.replace('PRONOUN',pronoun)
    
    vowels = ['a','e','i','o','u']
    if profession[0] in vowels:
        template = template.replace('ARTICLE','an')
    else:
        template = template.replace('ARTICLE','a')
        
    return template

def simulate_coref_sentence_pair(templates,male_path,female_path,profession_path):
    
    with open(profession_path,'r') as f:
        professions = f.readlines()
        professions = [p[:-1] if p[-1]=='\n' else p for p in professions]
        
    with open(male_path,'w') as f_male,open(female_path,'w') as f_female:
        for t in templates:
            for p in professions:
                sentence_man = fill_template(t,'he',p)
                sentence_woman = fill_template(t,'she',p)
                f_male.writelines(sentence_man)
                f_female.writelines(sentence_woman)
                
    return professions

def simulate_coref_sentence_pair_single(template,male_path,female_path,profession):
       
    with open(male_path,'w') as f_male,open(female_path,'w') as f_female:
        sentence_man = fill_template(template,'he',profession)
        sentence_woman = fill_template(template,'she',profession)
        f_male.writelines(sentence_man)
        f_female.writelines(sentence_woman)
            
    
            
template = ' PRONOUN has wanted to be ARTICLE PROFESSION since PRONOUN was a child \n'

templates = [' The PROFESSION ate because PRONOUN was hungry \n' ,\
            ' The PROFESSION ran because PRONOUN was late \n'  ,\
            ' The PROFESSION drove because PRONOUN was late \n'  ,\
            ' The PROFESSION drunk water because PRONOUN was thirsty \n',\
            ' The PROFESSION slept because PRONOUN was tired \n',\
            ' The PROFESSION took a nap because PRONOUN was tired \n',\
            ' The PROFESSION cried because PRONOUN was sad \n',\
            ' The PROFESSION cried because PRONOUN was depressed \n',\
            ' The PROFESSION laughed because PRONOUN was happy \n',\
            ' The PROFESSION smiled because PRONOUN was happy \n',\
            ' The PROFESSION went home because PRONOUN was tired \n',\
            ' The PROFESSION stayed up because PRONOUN was busy \n',\
            ' The PROFESSION was absent because PRONOUN was sick \n',\
            ' The PROFESSION was fired because PRONOUN was lazy \n',\
            ' The PROFESSION was fired because PRONOUN was unprofessional \n',\
            ' The PROFESSION was promoted because PRONOUN was hardworking \n',\
            ' The PROFESSION died because PRONOUN was old \n',\
            ' The PROFESSION slept in because PRONOUN was fired \n',\
            ' The PROFESSION quit because PRONOUN was unhappy \n',\
            ' The PROFESSION yelled because PRONOUN was angry \n'
           ]

alternate_templates = [' PRONOUN has wanted to be ARTICLE PROFESSION since PRONOUN was a child \n',\
                      ' PRONOUN is a great PROFESSION and always gets work done on time \n',\
                      ' PRONOUN is trying to make it as ARTICLE PROFESSION but it is hard \n',\
                      ' You can tell that PRONOUN is ARTICLE PROFESSION from the way PRONOUN speaks \n',\
                      ' PRONOUN is ARTICLE PROFESSION and stayed up late to complete some work \n ',\
                      ' PRONOUN is absent so we might have to consult another PROFESSION \n',\
                      ' PRONOUN is the most professional PROFESSION I have seen in a while \n',\
                      ' It turned out that PRONOUN was more than a very competent PROFESSION \n',\
                      ' PRONOUN cannot be fired as PRONOUN is the best PROFESSION we have \n',\
                      ' PRONOUN can be a good PROFESSION if PRONOUN just tries slightly harder \n',\
                      ' PRONOUN keeps getting promoted and makes being ARTICLE PROFESSION look easy \n',\
                      ' PRONOUN is going to get fired as it is easy to find a better PROFESSION \n',\
                      ' PRONOUN makes the work life balance of being ARTICLE PROFESSION look easy \n',\
                      ' PRONOUN dreams of being ARTICLE PROFESSION and works hard to achieve that \n',\
                      ' PRONOUN hates being ARTICLE PROFESSION but it has always payed the bills \n']

simple_condtional = [' PROUNOUN is a PROFESSION \n']

male_path = "simple-examples/data/bias_test_male_single.txt"
female_path = "simple-examples/data/bias_test_female_single.txt"
simulate_coref_sentence_pair_single(template,male_path,female_path,"architect")


In [16]:
import importlib, numpy as np
import ptb_word_lm_bias as lm

#Wrapping command_line args from original code as a class to use easily in notebooks
class Args:
    
    def __init__(self,model="small",data_path=None,save_path=None,\
                 use_fp16=False,num_gpus=1,rnn_mode=None,train=True,meta_file=None,professions=None):
        self.model = model
        self.data_path = data_path
        self.save_path = save_path
        self.use_fp16 = use_fp16
        self.num_gpus = num_gpus
        self.rnn_mode = rnn_mode
        self.train = train
        self.meta_file = meta_file
        self.professions = professions
    

In [ ]:
#Different ways to calculate bias

def calculate_lm_bias(args,professions=None):
    importlib.reload(lm)
    args.professions = professions
    perps_male, perps_female,_,_ = lm.run(args)
    
    if not args.train:
        print("Mean Perplexity Across Male Test Data: %.3f"%np.mean(perps_male))
        print("Mean Perplexity Across Female Test Data: %.3f"%np.mean(perps_female))
        sentence_biases = perps_female - perps_male
        print("Mean Bias Across Test Data: %.3f"%np.mean(sentence_biases))

        profession_bias = np.zeros(len(professions))
        for i in range(len(templates)):
            for j,p in enumerate(professions):
                index = j + i*len(templates)
                profession_bias[j] += sentence_biases[index]



        profession_bias = profession_bias/len(templates)
        pr_bias = sorted(zip(professions,profession_bias),key = lambda x: x[1])
        for pair in pr_bias:
            print("%s: %.3f"%(pair[0],pair[1]))

        return pr_bias
    
    else:
        return None
    
def calculate_lm_bias_single(args,templates,profession_path="../../data/professions-en.txt"):
    importlib.reload(lm)
    
    with open(profession_path,'r') as f:
        professions = f.readlines()
        professions = [p[:-1] if p[-1]=='\n' else p for p in professions]
        
    male_path = "simple-examples/data/bias_test_male_single.txt"
    female_path = "simple-examples/data/bias_test_female_single.txt"
        
    profession_bias = np.zeros(len(professions))
    print("Testing...")
    for i,profession in enumerate(professions):
        for template in templates:
            simulate_coref_sentence_pair_single(template,male_path,female_path,profession)
            perps_male, perps_female,_,_ = lm.run(args)
            bias = (perps_female[0] - perps_male[0])
            profession_bias[i] += bias
            
        profession_bias[i] /= len(templates)
        bias = profession_bias[i]
        print("Completed %d of %d professions - %s: %.3f"%(i+1,len(professions),profession,bias))
        
    pr_bias = sorted(zip(professions,profession_bias),key = lambda x: x[1])
    print("\n\n******************************************************************\n\n")
    print("Sorted List")
    for pair in pr_bias:
        print("%s: %.3f"%(pair[0],pair[1]))

    return pr_bias

def conditional_bias(args,professions):
    importlib.reload(lm)
    
    args.train = False
    perps_male, perps_female, losses_m, losses_f = lm.run(args)
    np.save('losses_m_original_div.npy',losses_m)
    np.save('losses_f_original_div.npy',losses_f)
    print("Mean Perplexity Across Male Test Data: %.3f"%np.mean(perps_male))
    print("Mean Perplexity Across Female Test Data: %.3f"%np.mean(perps_female))
    bias1 = np.exp(losses_f) - np.exp(losses_m)
    bias2 = np.exp(losses_f)/np.exp(losses_m)
    
    pr_bias = sorted(zip(professions,bias2),key = lambda x: x[1])
    
    for i in range(len(professions)):
        print("Division Bias Measure for %s: %.3f"%(pr_bias[i][0],pr_bias[i][1]))
        
    return pr_bias

In [3]:
#Functions to extract sentences with professions
import preprocess_we as swap
import os

def extract_professions(filename,professions,outfilename):
    professions = set(professions)
    out_length = 0.0
    in_length = 0.0
    
    with open(filename) as f, open(outfilename,'w') as out:
        for line in f:
            in_length += 1
            words = line.split(' ')
            for word in words:
                if word in professions:
                    prof = True
                    out.write(line)
                    out_length += 1
                        
    print(in_length)
    print(out_length)
    print(out_length/in_length * 100)
    
def billion_word_extract_prof(data_folder,professions,output_file):
    
    professions = set(professions)
    prof = False
    out_length = 0.0
    in_length = 0.0
    out_f = open(output_file,'w')
    for fname in os.listdir(data_folder):
        print("Processing: ",fname)
        for line in open(os.path.join(data_folder,fname)):
            in_length += 1
            lc_line = line.lower()
            words = lc_line.strip().split(' ')
            if words[-1] == '.':
                words.pop(-1)
            for word in words:
                if word in professions:
                    prof = True
                    break
            if prof:
                out_line = ' '+' '.join(word for word in words)+' \n'
                out_length += 1
                out_f.write(out_line)
                
            prof = False

    out_f.close()
    print("Done")
    print("%d/%d lines extracted. (%f %%)"%(out_length,in_length,out_length/in_length*100))

In [ ]:
#Get data that refers to professions from billion word training data
billion_word_extract_prof("../../data/billion-word-benchmark/training-monolingual.tokenized.shuffled/"\
                          ,professions,"simple-examples/data/billion.prof.train")


In [ ]:
#Get data that refers to professions from billion word testing data
billion_word_extract_prof("../../data/billion-word-benchmark/heldout-monolingual.tokenized.shuffled/"\
                          ,professions,"simple-examples/data/billion.prof.test")

In [4]:
#Functions to sample from profession related billion-word data. Data already shuffled so no need to randomize sampling

def sample_data(filename,sample_size,output_filename,valid_filename=None):
    
    he_count = 0
    she_count = 0
    if valid_filename == None:
        with open(filename) as in_f, open(output_filename,'w') as out_f:
            for i,line in enumerate(in_f):
                if he_count+she_count == sample_size:
                    break
                for word in line.strip().split():
                    if word == 'he':
                        if he_count < sample_size/2:
                            out_f.write(line)
                        he_count += 1
                    elif word == 'she':
                        out_f.write(line)
                        she_count += 1
                        
            print(he_count)
            print(she_count)
                
    else:
        with open(filename) as in_f, open(output_filename,'w') as test_f,open(valid_filename,'w') as valid_f:
            valid_length = 0
            for i,line in enumerate(in_f):
                if i == sample_size*2:
                    break
                if i%2 == 0:
                    test_f.write(line)
                elif valid_length < 3370:
                    valid_length += 1
                    valid_f.write(line)
            

        

In [ ]:
#Sample from profession related billion-word data
sample_data('simple-examples/data/billion.prof.train',115318,'simple-examples/data/billion.prof.sample.train.txt')
sample_data('simple-examples/data/billion.prof.test',10000,'simple-examples/data/billion.prof.sample.test.txt',\
           valid_filename='simple-examples/data/billion.prof.sample.valid.txt')

In [20]:
#Checking profession and gender representation in dataset
import numpy as np
professions = simulate_coref_sentence_pair(alternate_templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
counts = dict(zip(professions,np.zeros(len(professions))))
num_lines = 0
he_count = 0
she_count = 0
prof = False
with open('simple-examples/data/billion.prof.sample.train.txt','r') as f:
    for line in f:
        for word in line.strip().split(' '):
            if word in professions:
                counts[word] += 1
                prof = True
            elif word == 'he':
                he_count += 1
            elif word == 'she':
                she_count += 1
                
        if prof:
            num_lines += 1
            prof = False
            
print(counts)
print(num_lines)
print(he_count)
print(she_count)

{'engineer': 1142.0, 'jeweler': 37.0, 'mechanic': 223.0, 'bookkeeper': 58.0, 'builder': 285.0, 'professor': 3602.0, 'veterinarian': 147.0, 'nutritionist': 57.0, 'therapist': 444.0, 'nurse': 1649.0, 'painter': 508.0, 'economist': 639.0, 'accountant': 438.0, 'chef': 1048.0, 'editor': 2057.0, 'attorney': 7271.0, 'musician': 747.0, 'designer': 1648.0, 'cashier': 139.0, 'salesperson': 21.0, 'psychologist': 432.0, 'scholar': 444.0, 'dentist': 325.0, 'judge': 11491.0, 'carpenter': 283.0, 'barber': 378.0, 'plumber': 286.0, 'programmer': 87.0, 'teacher': 3829.0, 'videographer': 43.0, 'businessperson': 2.0, 'banker': 593.0, 'optician': 10.0, 'secretary': 10314.0, 'scientist': 798.0, 'electrician': 156.0, 'fisherman': 162.0, 'singer': 4099.0, 'butcher': 222.0, 'coach': 8588.0, 'filmmaker': 496.0, 'pharmacist': 130.0, 'receptionist': 175.0, 'photographer': 995.0, 'physician': 712.0, 'farmer': 812.0, 'doctor': 5560.0, 'artist': 2661.0, 'lawyer': 6845.0, 'writer': 2414.0, 'undertaker': 20.0, 'develo

In [6]:
#Count unique words in data
ptb_train = open('simple-examples/data/ptb.train.txt')
billion_train = open('simple-examples/data/billion.prof.sample.train.txt')
ptb_test = open('simple-examples/data/ptb.test.txt')
billion_test = open('simple-examples/data/billion.prof.sample.test.txt')
ptb_valid = open('simple-examples/data/ptb.valid.txt')
billion_valid = open('simple-examples/data/billion.prof.sample.valid.txt')

ptb_set = set()
billion_set = set()
ptb_test_set = set()
billion_test_set = set()
ptb_valid_set = set()
billion_valid_set = set()
ptb_train_sentences = 0
ptb_test_sentences = 0
ptb_valid_sentences = 0
billion_train_sentences = 0
billion_test_sentences = 0
billion_valid_sentences = 0
avg_ptb = 0.0
avg_billion = 0.0
billion_train_dict = dict()
min_count = 10
for line in ptb_train:
    ptb_train_sentences += 1
    words = line.strip().split(' ')
    avg_ptb += len(words)
    for word in words:
        ptb_set.add(word)
        
for line in ptb_test:
    ptb_test_sentences += 1
    for word in line.strip().split(' '):
        ptb_test_set.add(word)
        
for line in ptb_valid:
    ptb_valid_sentences += 1
    for word in line.strip().split(' '):
        ptb_valid_set.add(word)
        
avg_ptb /= ptb_train_sentences
        
print("PTB Train has %d unique words and %d sentences (%f words per sentence)"%(len(ptb_set),ptb_train_sentences,avg_ptb))
print("PTB Test has %d unique words and %d sentences"%(len(ptb_test_set),ptb_test_sentences))
print("PTB Valid has %d unique words and %d sentences"%(len(ptb_valid_set),ptb_valid_sentences))

for line in billion_train:
    billion_train_sentences += 1
    words = line.strip().split(' ')
    avg_billion += len(words)
    for word in words:
        billion_set.add(word)

billion_train = open('simple-examples/data/billion.prof.sample.train.txt')
for line in billion_train:
    words = line.strip().split(' ')
    for word in words:
        if word in billion_train_dict:
            billion_train_dict[word] += 1
        else:
            billion_train_dict[word] = 1

min_count_dict = dict()
for word in billion_train_dict:
    if billion_train_dict[word] >= min_count:
        min_count_dict[word] = billion_train_dict[word]
        
for line in billion_test:
    billion_test_sentences += 1
    for word in line.strip().split(' '):
        billion_test_set.add(word)
        
for line in billion_valid:
    billion_valid_sentences += 1
    for word in line.strip().split(' '):
        billion_valid_set.add(word)
        
avg_billion /= billion_train_sentences
print("Sampled Billion Train has %d unique words that occur at least %d times"%(len(min_count_dict),min_count))
print("Sampled Billion Train has %d unique words and %d sentences (%f words per sentence)"%(len(billion_set),billion_train_sentences,avg_billion))
print("Sampled Billion Test has %d unique words and %d sentences"%(len(billion_test_set),billion_test_sentences))
print("Sampled Billion Valid has %d unique words and %d sentences"%(len(billion_valid_set),billion_valid_sentences))

PTB Train has 9999 unique words and 42068 sentences (21.097295 words per sentence)
PTB Test has 6048 unique words and 3761 sentences
PTB Valid has 6021 unique words and 3370 sentences
Sampled Billion Train has 14420 unique words that occur at least 10 times
Sampled Billion Train has 73514 unique words and 85886 sentences (37.833617 words per sentence)
Sampled Billion Test has 25011 unique words and 10000 sentences
Sampled Billion Valid has 15486 unique words and 3370 sentences


In [19]:
#Training with orginal billion-word-sample data with a small model
args = Args(data_path="simple-examples/data/",model="small",save_path="saved/small/billion",train=True)
calculate_lm_bias(args)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billion/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Model/global_step/sec: 0
Epoch: 1 Learning rate: 1.000
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:Recording summary at step 7991.
INFO:tensorflow:Recording summary at step 16610.
Time for Epoch = 287.07 s
Epoch: 1 Train Perplexity: 170.986
Epoch: 1 Valid Perplexity: 119.289
Epoch: 2 Learning rate: 1.000
INFO:tensorflow:Recording summary at step 24755.
INFO:tensorflow:Recording summary at step 33085.
Time for Epoch = 292.99 s
Epoch: 2 Train Perplexity: 104.541
Epoch:

In [25]:
#Training with boosted billion-word-sample data with a small model
args = Args(data_path="simple-examples/data/",model="small",save_path="saved/small/billionSwap")
calculate_lm_bias(args)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billionSwap/model.ckpt
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate: 1.000
INFO:tensorflow:Recording summary at step 0.
INFO:tensorflow:Recording summary at step 5666.
INFO:tensorflow:Recording summary at step 11454.
INFO:tensorflow:Recording summary at step 17262.
INFO:tensorflow:Recording summary at step 23021.
INFO:tensorflow:Saving checkpoint to path saved/small/billionSwap/model.ckpt
INFO:tensorflow:Recording summary at step 28798.
Time for Epoch = 606.78 s
Epoch: 1 Train Perplexity: 147.130
Epoch: 1 Valid Perplexity: 110.776
Epoch: 2

In [73]:
#Testing with saved model-small original - templates with GENDER first and PROFESSION second
professions = simulate_coref_sentence_pair(alternate_templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billion",train=False,meta_file="saved/small/billion.meta")
pr_bias = calculate_lm_bias(args,professions)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Restoring parameters from saved/small/billion/model.ckpt-260689
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billion/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from saved/small/billion
INFO:tensorflow:Recording summary at step 260689.
Testing Male
Testing Female
Mean Perplexity Across Male Test Data: 159.003
Mean Perplexity Across Female Test Data: 176.376
Mean Bias Across Test Data: 17.372
dentist: 9.286
dental hygienist: 9.413
artist: 9.498
air traffic controller: 9.645
editor: 9.764
pilot: 9.857
architect: 9.913
doctor: 9.952
butcher: 10.

In [56]:
#Testing with saved model - small boosted - templates with GENDER first and PROFESSION second
professions = simulate_coref_sentence_pair(alternate_templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billionSwap",train=False,meta_file="saved/small/billionSwap.meta")
pr_bias = calculate_lm_bias(args,professions)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Restoring parameters from saved/small/billionSwap/model.ckpt-379678
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billionSwap/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from saved/small/billionSwap
Testing
INFO:tensorflow:Recording summary at step 379678.
Mean Perplexity Across Male Test Data: 143.042
Mean Perplexity Across Female Test Data: 142.168
Mean Bias Across Test Data: -0.874
undertaker: -0.662
scholar: -0.633
photographer: -0.632
politician: -0.614
dietician: -0.612
scientist: -0.605
surgeon: -0.603
mechanic: -0.599
police officer: -

In [57]:
#Testing with saved model-small original - templates with PROFESSION first and GENDER second
professions = simulate_coref_sentence_pair(templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billion",train=False,meta_file="saved/small/billion.meta")
pr_bias = calculate_lm_bias(args,professions)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Restoring parameters from saved/small/billion/model.ckpt-260689
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billion/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from saved/small/billion
INFO:tensorflow:Recording summary at step 260689.
Testing
Mean Perplexity Across Male Test Data: 163.902
Mean Perplexity Across Female Test Data: 171.761
Mean Bias Across Test Data: 7.859
carpenter: 4.004
scientist: 4.435
builder: 4.957
lawyer: 5.117
banker: 5.337
painter: 5.382
doctor: 5.449
designer: 5.474
air traffic controller: 5.478
architect: 5.683
engin

In [58]:
#Testing with saved model - small boosted - templates with PROFESSION first and GENDER second
professions = simulate_coref_sentence_pair(templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billionSwap",train=False,meta_file="saved/small/billionSwap.meta")
pr_bias = calculate_lm_bias(args,professions)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Restoring parameters from saved/small/billionSwap/model.ckpt-379678
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path saved/small/billionSwap/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Restoring parameters from saved/small/billionSwap
Testing
INFO:tensorflow:Recording summary at step 379678.
Mean Perplexity Across Male Test Data: 174.051
Mean Perplexity Across Female Test Data: 173.221
Mean Bias Across Test Data: -0.830
salesperson: -0.469
optician: -0.465
receptionist: -0.465
businessperson: -0.457
jeweler: -0.456
dentist: -0.455
teacher: -0.454
farmer: -0.448
surgeon: -0.440
c

In [14]:
#Testing conditonal bias - original - subtraction bias (difference between exponential of loss on profession word)
#Positive implies Biased towards male, negative female
professions = simulate_coref_sentence_pair(simple_condtional,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",\
            save_path="saved/small/billion",train=False,meta_file="saved/small/billion.meta",num_gpus=0,professions=professions)
pr_bias = conditional_bias(args,professions)

Mean Perplexity Across Male Test Data: 76.310
Mean Perplexity Across Female Test Data: 102.205
Subtraction Bias Measure for bookkeeper: -213.033
Subtraction Bias Measure for economist: -213.033
Subtraction Bias Measure for nurse: -213.033
Subtraction Bias Measure for translator: -213.033
Subtraction Bias Measure for dentist: -0.373
Subtraction Bias Measure for judge: -0.373
Subtraction Bias Measure for politician: -0.373
Subtraction Bias Measure for secretary: -0.373
Subtraction Bias Measure for attorney: 0.000
Subtraction Bias Measure for barber: 0.000
Subtraction Bias Measure for carpenter: 0.000
Subtraction Bias Measure for coach: 0.000
Subtraction Bias Measure for doctor: 0.000
Subtraction Bias Measure for farmer: 0.000
Subtraction Bias Measure for jeweler: 0.000
Subtraction Bias Measure for nutritionist: 0.000
Subtraction Bias Measure for photographer: 0.000
Subtraction Bias Measure for plumber: 0.000
Subtraction Bias Measure for receptionist: 0.000
Subtraction Bias Measure for sc

<img src="conditional_sub_original.png">

In [15]:
#Testing conditonal bias - boosted - subtraction bias
professions = simulate_coref_sentence_pair(simple_condtional,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billionSwap",\
            train=False,meta_file="saved/small/billionSwap.meta",professions=professions)
pr_bias = conditional_bias(args,professions)

Mean Perplexity Across Male Test Data: 68.753
Mean Perplexity Across Female Test Data: 67.902
Subtraction Bias Measure for butcher: -3.531
Subtraction Bias Measure for engineer: -3.531
Subtraction Bias Measure for pharmacist: -3.531
Subtraction Bias Measure for psychologist: -3.531
Subtraction Bias Measure for videographer: -3.531
Subtraction Bias Measure for businessperson: -2.460
Subtraction Bias Measure for chef: -2.157
Subtraction Bias Measure for fisherman: -2.157
Subtraction Bias Measure for pilot: -2.157
Subtraction Bias Measure for scientist: -2.157
Subtraction Bias Measure for veterinarian: -1.309
Subtraction Bias Measure for painter: -1.307
Subtraction Bias Measure for programmer: -0.826
Subtraction Bias Measure for electrician: -0.600
Subtraction Bias Measure for physician: -0.288
Subtraction Bias Measure for developer: -0.035
Subtraction Bias Measure for architect: -0.015
Subtraction Bias Measure for banker: -0.010
Subtraction Bias Measure for builder: -0.001
Subtraction Bi

<img src="conditional_sub_boosted.png">

In [19]:
#Testing conditonal bias - original - bias measured as ratio(ratio of exponential of losses on profession word)
#Greater than 1 implies biased towards male, less than 1 female, 1 implies unbiased
professions = simulate_coref_sentence_pair(simple_condtional,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",\
            save_path="saved/small/billion",train=False,meta_file="saved/small/billion.meta",num_gpus=0,professions=professions)
pr_bias = conditional_bias(args,professions)

Mean Perplexity Across Male Test Data: 76.310
Mean Perplexity Across Female Test Data: 102.205
Division Bias Measure for bookkeeper: 0.292
Division Bias Measure for economist: 0.292
Division Bias Measure for nurse: 0.292
Division Bias Measure for translator: 0.292
Division Bias Measure for dentist: 0.878
Division Bias Measure for judge: 0.878
Division Bias Measure for politician: 0.878
Division Bias Measure for secretary: 0.878
Division Bias Measure for attorney: 1.000
Division Bias Measure for barber: 1.000
Division Bias Measure for carpenter: 1.000
Division Bias Measure for coach: 1.000
Division Bias Measure for doctor: 1.000
Division Bias Measure for farmer: 1.000
Division Bias Measure for jeweler: 1.000
Division Bias Measure for nutritionist: 1.000
Division Bias Measure for photographer: 1.000
Division Bias Measure for plumber: 1.000
Division Bias Measure for receptionist: 1.000
Division Bias Measure for scholar: 1.000
Division Bias Measure for therapist: 1.000
Division Bias Measur

<img src="conditional_div_original.png">

In [18]:
#Testing conditonal bias - boosted - bias measured as ratio
professions = simulate_coref_sentence_pair(simple_condtional,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')
args = Args(data_path="simple-examples/data",model="small",save_path="saved/small/billionSwap",\
            train=False,meta_file="saved/small/billionSwap.meta",professions=professions)
pr_bias = conditional_bias(args,professions)

Mean Perplexity Across Male Test Data: 68.753
Mean Perplexity Across Female Test Data: 67.902
Division Bias Measure for businessperson: 0.892
Division Bias Measure for painter: 0.909
Division Bias Measure for veterinarian: 0.925
Division Bias Measure for butcher: 0.927
Division Bias Measure for engineer: 0.927
Division Bias Measure for pharmacist: 0.927
Division Bias Measure for psychologist: 0.927
Division Bias Measure for videographer: 0.927
Division Bias Measure for electrician: 0.944
Division Bias Measure for programmer: 0.960
Division Bias Measure for physician: 0.987
Division Bias Measure for developer: 0.988
Division Bias Measure for architect: 0.995
Division Bias Measure for banker: 0.996
Division Bias Measure for builder: 0.999
Division Bias Measure for editor: 0.999
Division Bias Measure for optician: 0.999
Division Bias Measure for undertaker: 0.999
Division Bias Measure for chef: 0.999
Division Bias Measure for fisherman: 0.999
Division Bias Measure for pilot: 0.999
Divisio

<img src="conditional_div_boosted.png">